## Inform experiment using DMP Results
Check the DMP results dataset to get experimental parameters.

Query: What is the epoch, depth, and budget that leads to the best validation loss for MNIST and 537_houses?

In [ ]:
import sys
sys.path.append("../../../DMP-viz/2021-fixed-3k-summary/")

In [14]:
from viz_utils import *

In [69]:
params_premap = [
    ('batch', 'fixed_3k_1'),
    ('learning_rate', 0.0001),
    ('label_noise', 0.0),
    ('dataset', "mnist"),
    ('shape', "rectangle")
]

param_map = parameter_map.to_parameter_ids(params_premap)

query_string = f'''
SELECT * from (
    SELECT DISTINCT ON (parameters)
        parameters,
        a.val as min_avg_val_loss,
        a.epoch
    FROM
        experiment_summary,
        unnest(val_loss_avg) with ordinality as a(val, epoch)
    WHERE
        parameters @> (array[{ param_map }])::smallint[]
    ORDER BY parameters, min_avg_val_loss ASC
    ) as foo
ORDER BY foo.min_avg_val_loss limit 20;
'''

In [70]:
df = query(query_string)

kinds = [
('batch', 'str'),
('dataset', 'str'),
('learning_rate', 'float'),
('label_noise', 'float'),
('shape', 'str'),
('depth', 'int'),
('size', 'int'),
]

print('Interpeting parameters...')
def make_parameter_map(parameter_id_list):
    m = {kind : value for kind, value in parameter_map.parameter_from_id(parameter_id_list)}
    return tuple((m.get(kind[0], None) for kind in kinds))

parameters = list(zip(*df['parameters'].map(make_parameter_map)))

print('Converting parameters...')
for i, (kind, typename) in enumerate(kinds):
    df[kind] = parameters[i]
    df[kind] = df[kind].astype(typename)  

df.drop(columns=['parameters'], inplace=True)

Interpeting parameters...
Converting parameters...


In [71]:
df

,min_avg_val_loss,epoch,batch,dataset,learning_rate,label_noise,shape,depth,size
0,0.061263,20,fixed_3k_1,mnist,0.0001,0.0,rectangle,2,16777216
1,0.062823,26,fixed_3k_1,mnist,0.0001,0.0,rectangle,2,4194304
2,0.064110,22,fixed_3k_1,mnist,0.0001,0.0,rectangle,2,8388608
3,0.064781,35,fixed_3k_1,mnist,0.0001,0.0,rectangle,2,2097152
4,0.066607,9,fixed_3k_1,mnist,0.0001,0.0,rectangle,3,16777216
5,0.066837,44,fixed_3k_1,mnist,0.0001,0.0,rectangle,2,1048576
6,0.067622,12,fixed_3k_1,mnist,0.0001,0.0,rectangle,3,8388608
7,0.068828,17,fixed_3k_1,mnist,0.0001,0.0,rectangle,3,2097152
8,0.069466,60,fixed_3k_1,mnist,0.0001,0.0,rectangle,2,524288
9,0.069625,12,fixed_3k_1,mnist,0.0001,0.0,rectangle,3,4194304
